In [1]:
import h5py
import pandas as pd
import shutil
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

2023-05-31 19:58:19.253769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 19:58:19.816907: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-31 19:58:19.816959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-31 19:58:19.816963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(10*1000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
TRAIN_OLD = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_training.hdf5'
VAL_OLD = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_validation.hdf5'
TEST_OLD = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_testing.hdf5'
TRAIN_NEW = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_training_with_morphology_normalized.hdf5'
VAL_NEW = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_validation_with_morphology_normalized.hdf5'
TEST_NEW = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_testing_with_morphology_normalized.hdf5'
with h5py.File(TRAIN_OLD, 'r') as f:
    shutil.copyfile(TRAIN_OLD, TRAIN_NEW)
with h5py.File(VAL_OLD, 'r') as f:
    shutil.copyfile(VAL_OLD, VAL_NEW)
with h5py.File(TEST_OLD, 'r') as f:
    shutil.copyfile(TEST_OLD, TEST_NEW)

In [4]:
g_df = pd.read_csv('/data/HSC/HSC_v6/step1/g_band_sextractor/shape_parameters.csv')
r_df = pd.read_csv('/data/HSC/HSC_v6/step1/r_band_sextractor/shape_parameters.csv')
i_df = pd.read_csv('/data/HSC/HSC_v6/step1/i_band_sextractor/shape_parameters.csv')
z_df = pd.read_csv('/data/HSC/HSC_v6/step1/z_band_sextractor/shape_parameters.csv')
y_df = pd.read_csv('/data/HSC/HSC_v6/step1/y_band_sextractor/shape_parameters.csv')

In [5]:
g_df = g_df[['object_id', 'specz', 'g_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN']]
r_df = r_df[['object_id', 'specz', 'r_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN']]
i_df = i_df[['object_id', 'specz', 'i_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN']]
z_df = z_df[['object_id', 'specz', 'z_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN']]
y_df = y_df[['object_id', 'specz', 'y_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN']]

In [6]:
g_df.rename(columns={'PETRO_RADIUS': 'g_petro_rad', 'ISOAREA_IMAGE': 'g_isophotal_area', 'A_IMAGE': 'g_major_axis', 'B_IMAGE': 'g_minor_axis', 'THETA_IMAGE': 'g_pos_angle', 'MU_MAX': 'g_peak_surface_brightness', 'ELLIPTICITY': 'g_ellipticity', 'FLUX_RADIUS': 'g_half_light_radius', 'SPHEROID_SERSICN': 'g_sersic_index'}, inplace=True)
r_df.rename(columns={'PETRO_RADIUS': 'r_petro_rad', 'ISOAREA_IMAGE': 'r_isophotal_area', 'A_IMAGE': 'r_major_axis', 'B_IMAGE': 'r_minor_axis', 'THETA_IMAGE': 'r_pos_angle', 'MU_MAX': 'r_peak_surface_brightness', 'ELLIPTICITY': 'r_ellipticity', 'FLUX_RADIUS': 'r_half_light_radius', 'SPHEROID_SERSICN': 'r_sersic_index'}, inplace=True)
i_df.rename(columns={'PETRO_RADIUS': 'i_petro_rad', 'ISOAREA_IMAGE': 'i_isophotal_area', 'A_IMAGE': 'i_major_axis', 'B_IMAGE': 'i_minor_axis', 'THETA_IMAGE': 'i_pos_angle', 'MU_MAX': 'i_peak_surface_brightness', 'ELLIPTICITY': 'i_ellipticity', 'FLUX_RADIUS': 'i_half_light_radius', 'SPHEROID_SERSICN': 'i_sersic_index'}, inplace=True)
z_df.rename(columns={'PETRO_RADIUS': 'z_petro_rad', 'ISOAREA_IMAGE': 'z_isophotal_area', 'A_IMAGE': 'z_major_axis', 'B_IMAGE': 'z_minor_axis', 'THETA_IMAGE': 'z_pos_angle', 'MU_MAX': 'z_peak_surface_brightness', 'ELLIPTICITY': 'z_ellipticity', 'FLUX_RADIUS': 'z_half_light_radius', 'SPHEROID_SERSICN': 'z_sersic_index'}, inplace=True)
y_df.rename(columns={'PETRO_RADIUS': 'y_petro_rad', 'ISOAREA_IMAGE': 'y_isophotal_area', 'A_IMAGE': 'y_major_axis', 'B_IMAGE': 'y_minor_axis', 'THETA_IMAGE': 'y_pos_angle', 'MU_MAX': 'y_peak_surface_brightness', 'ELLIPTICITY': 'y_ellipticity', 'FLUX_RADIUS': 'y_half_light_radius', 'SPHEROID_SERSICN': 'y_sersic_index'}, inplace=True)

In [7]:
df = pd.merge(g_df, r_df, on=['object_id', 'specz'])
df = pd.merge(i_df, df, on=['object_id', 'specz'])
df = pd.merge(z_df, df, on=['object_id', 'specz'])
df = pd.merge(y_df, df, on=['object_id', 'specz'])

In [8]:
df

,object_id,specz,y_cmodel_mag,y_petro_rad,y_isophotal_area,y_major_axis,y_minor_axis,y_pos_angle,y_peak_surface_brightness,y_ellipticity,...,r_cmodel_mag,r_petro_rad,r_isophotal_area,r_major_axis,r_minor_axis,r_pos_angle,r_peak_surface_brightness,r_ellipticity,r_half_light_radius,r_sersic_index
0,36407046198803509,0.43602,17.614677,7.92,372.0,4.266,3.966,-82.86,-7.8749,0.070,...,18.924778,6.60,673.0,5.447,4.986,72.23,-6.4936,0.085,6.856,1.502
1,36407046198804043,0.23209,16.851135,7.92,824.0,6.518,5.251,-38.78,-8.4882,0.194,...,18.092823,7.26,1596.0,9.150,7.210,-39.48,-7.0298,0.212,10.747,2.224
2,36407050493759629,0.56511,18.164425,9.24,399.0,4.771,3.885,-87.73,-7.3739,0.186,...,19.881702,8.58,393.0,4.814,3.993,-85.69,-5.4887,0.171,7.051,1.620
3,36407050493773284,0.55775,19.115318,9.90,190.0,3.286,2.800,61.63,-7.1324,0.148,...,20.791992,10.56,197.0,3.546,2.854,71.63,-5.3227,0.195,5.932,2.257
4,36407054788744371,0.44495,18.476147,6.60,233.0,3.420,3.023,-39.18,-7.9104,0.116,...,19.897354,5.94,380.0,3.928,3.836,-47.82,-6.3640,0.023,4.704,1.359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286396,74649168714401583,1.86393,20.932972,5.94,62.0,1.786,1.721,85.12,-6.6126,0.036,...,21.244419,5.28,131.0,2.232,2.069,62.51,-6.1867,0.073,2.394,1.041
286397,74649168714403940,0.89864,21.601278,9.90,34.0,1.875,1.305,-28.82,-5.0249,0.304,...,22.500662,6.60,81.0,2.398,2.007,-16.20,-4.0196,0.163,3.281,0.550
286398,74649168714420584,0.24194,16.965544,6.60,972.0,6.226,5.502,-52.62,-8.7467,0.116,...,18.017313,5.94,1229.0,6.638,5.815,-45.89,-7.7220,0.124,6.367,1.995
286399,74649168714426572,0.77584,21.360008,10.56,17.0,1.639,0.817,-88.91,-4.7957,0.501,...,22.201458,6.60,111.0,3.169,2.163,67.10,-3.9291,0.318,4.182,0.558


In [9]:
df.drop(columns='specz', inplace=True)

In [10]:
with h5py.File(TRAIN_NEW, 'r') as f:
    train_oid = list(f['object_id'][:])
train_oid_df = pd.DataFrame({'object_id': train_oid, 'index_pos': range(len(train_oid))})
merged_train = pd.merge(df, train_oid_df, on='object_id', how='inner')
sorted_train = merged_train.sort_values('index_pos')
sorted_train = sorted_train.drop(columns=['object_id', 'g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag', 'index_pos'], axis=1)
train_scaler = MinMaxScaler()
normalized_train = train_scaler.fit_transform(sorted_train.values)

In [11]:
with h5py.File(VAL_NEW, 'r') as f:
    val_oid = list(f['object_id'][:])
val_oid_df = pd.DataFrame({'object_id': val_oid, 'index_pos': range(len(val_oid))})
merged_val = pd.merge(df, val_oid_df, on='object_id', how='inner')
sorted_val = merged_val.sort_values('index_pos')
sorted_val = sorted_val.drop(columns=['object_id', 'g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag', 'index_pos'], axis=1)
val_scaler = MinMaxScaler()
normalized_val = val_scaler.fit_transform(sorted_val.values)

In [12]:
with h5py.File(TEST_NEW, 'r') as f:
    test_oid = list(f['object_id'][:])
test_oid_df = pd.DataFrame({'object_id': test_oid, 'index_pos': range(len(test_oid))})
merged_test = pd.merge(df, test_oid_df, on='object_id', how='inner')
sorted_test = merged_test.sort_values('index_pos')
sorted_test = sorted_test.drop(columns=['object_id', 'g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag', 'index_pos'], axis=1)
test_scaler = MinMaxScaler()
normalized_test = test_scaler.fit_transform(sorted_test.values)

In [13]:
with h5py.File(TRAIN_NEW, 'a') as f:
    column_names = sorted_train.columns.tolist()
    for i, column_name in enumerate(column_names):
        column_data = normalized_train[:, i]
        f[column_name] = column_data

In [14]:
with h5py.File(VAL_NEW, 'a') as f:
    column_names = sorted_val.columns.tolist()
    for i, column_name in enumerate(column_names):
        column_data = normalized_val[:, i]
        f[column_name] = column_data

In [15]:
with h5py.File(TEST_NEW, 'a') as f:
    column_names = sorted_test.columns.tolist()
    for i, column_name in enumerate(column_names):
        column_data = normalized_test[:, i]
        f[column_name] = column_data

In [16]:
with h5py.File(TEST_NEW, 'a') as f:
    print(list(f.keys()))

['coord', 'dec', 'g_cmodel_mag', 'g_cmodel_magsigma', 'g_ellipticity', 'g_half_light_radius', 'g_isophotal_area', 'g_major_axis', 'g_minor_axis', 'g_peak_surface_brightness', 'g_petro_rad', 'g_pos_angle', 'g_sersic_index', 'i_cmodel_mag', 'i_cmodel_magsigma', 'i_ellipticity', 'i_half_light_radius', 'i_isophotal_area', 'i_major_axis', 'i_minor_axis', 'i_peak_surface_brightness', 'i_petro_rad', 'i_pos_angle', 'i_sersic_index', 'image', 'object_id', 'r_cmodel_mag', 'r_cmodel_magsigma', 'r_ellipticity', 'r_half_light_radius', 'r_isophotal_area', 'r_major_axis', 'r_minor_axis', 'r_peak_surface_brightness', 'r_petro_rad', 'r_pos_angle', 'r_sersic_index', 'ra', 'skymap_id', 'specz_dec', 'specz_flag_homogeneous', 'specz_mag_i', 'specz_name', 'specz_ra', 'specz_redshift', 'specz_redshift_err', 'y_cmodel_mag', 'y_cmodel_magsigma', 'y_ellipticity', 'y_half_light_radius', 'y_isophotal_area', 'y_major_axis', 'y_minor_axis', 'y_peak_surface_brightness', 'y_petro_rad', 'y_pos_angle', 'y_sersic_index'

In [17]:
# make this for the 5 bands?
# come up with names for the shape parameters if needed
# if this goes in the data, make others have input on this

In [18]:
# ISOAREA_IMAGE -> isophotal_area
# A_IMAGE -> major_axis
# B_IMAGE -> minor_axis
# FLUX_RADIUS -> half_light_rad
# PETRO_RADIUS -> petro_rad
# THETA_WORLD -> pos_angle
# SPHEROID_SERSICN -> sersic_index
# ELLIPTICITY -> ellipticity
# MU_MAX -> peak_surfance_brightness